[딥 러닝을 이용한 자연어 처리 입문](https://wikidocs.net/22650)을 참고하였습니다.

# 카운트를 기반으로 한 단어 표현방법을 알아보자.

## BoW
(Bag of Words)
- 단어 순서 고려하지 않고 빈도에만 집중

### 만드는 과정
1) 각 단어에 고유 인덱스 부여  
2) 각 인덱스 위치에 단어 토큰의 등장 횟수를 기록한 벡터 생성

# DTM
(Document-Term Matrix)
- 여러 문서에서 등장하는 각 단어들의 빈도를 행렬로 표현한 것(서로 다른 문서들의 BoW를 결합)
- 문서들을 수치화함으로써 다른 문서들과 비교할 수 있게 된다.

## 한계점
#### 1) 희소표현
- 각 문서벡터의 차원은 전체 단어 집합의 크기
    - 데이터가 크다면 대부분의 값이 0을 가지게 된다.
    - 단어집합의 크기를 줄이기 위해 구두점, 빈도수 낮은 단어, 불용어, 어간 표제어 추출 등 정규화를 이용
    
#### 2) 단순 빈도 수 기반 접근
- 의미는 없지만 빈도수가 높은 단어들이 있는 문서들을 유사하다고 판단할 수 있다.


따라서, 중요한 단어에 대해서 가중치를 줄 수 있는 방법(TF-IDF)에 대해서 알아보도록 하자.

# TF-IDF
(Term Frequency-Inverse Document Frequency)

- 단어 빈도(TF)와 역 문서 빈도(IDF)를 사용하여 DTM 내의 각 단어들마다 중요한 정도를 가중치로 주는 방법
- 모든 문서에서 자주 등장하는 단어는 중요도가 낮다고 판단 <-> 특정 문서에서만 등장하는 단어는 중요도가 높다고 판단.  

예) the, a와 같은 불용어는 중요도가 낮다.
- TF-IDF는 클수록 중요도가 높다.
    
### 활용
- 문서의 유사도 구하기
- 검색 결과의 중요도를 정하는 작업
- 문서 내 특정 단어의 중요도 구하는 작업

### 상세 설명
d: 문서  
t: 단어  
n: 문서 개수

#### 1) tf(d,t): 특정 문서 d에서의 특정 단어 t의 등장 횟수

#### 2) df(t): 특정 단어 t가 등장한 문서의 수

#### 3) idf(d,t): df(t)에 반비례하는 수

### 파이썬으로 TF-IDF 직접 구현하기

In [1]:
import pandas as pd
from math import log

In [2]:
docs = ['gold silver truck',
        'Shipment of gold damaged in a fire',
        'Delivery of silver arrived in a silver truck',
        'Shipment of gold arrived in a truck'
       ]

vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()

In [24]:
# TF
N =len(docs)

def tf(d, t):
    return d.count(t)

def idf(t):
    df = 0
    for doc in docs:
        df += t in doc
    return log(N/(df+1))

def tfidf(t,d):
    return tf(d,t)*idf(t)

In [26]:
result = []
for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]
        result[-1].append(tf(d,t))
        
tf_ = pd.DataFrame(result, columns = vocab)
tf_

,Delivery,Shipment,a,arrived,damaged,fire,gold,in,of,silver,truck
0,0,0,0,0,0,0,1,0,0,1,1
1,0,1,3,0,1,1,1,1,1,0,0
2,1,0,2,1,0,0,0,1,1,2,1
3,0,1,2,1,0,0,1,1,1,0,1


In [34]:
result = []
for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))
    
idf_ = pd.DataFrame(result, index = vocab, columns = ["IDF"])
idf_

,IDF
Delivery,0.693147
Shipment,0.287682
a,0.000000
arrived,0.287682
damaged,0.693147
fire,0.693147
gold,0.000000
in,0.000000
of,0.000000
silver,0.287682


In [38]:
result = []
for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]

        result[-1].append(tfidf(t,d))

tfidf_ = pd.DataFrame(result, columns = vocab)
tfidf_

,Delivery,Shipment,a,arrived,damaged,fire,gold,in,of,silver,truck
0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.287682,0.0
1,0.000000,0.287682,0.0,0.000000,0.693147,0.693147,0.0,0.0,0.0,0.000000,0.0
2,0.693147,0.000000,0.0,0.287682,0.000000,0.000000,0.0,0.0,0.0,0.575364,0.0
3,0.000000,0.287682,0.0,0.287682,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
